# Demo: AI Supply Assistant - Programmatic Access

Ten notebook pokazuje, jak używać modułów `DatabaseConnector` i `Forecaster` bezpośrednio w Pythonie, co jest przydatne do automatyzacji raportów lub eksperymentów data science.

In [ ]:
import pandas as pd
import sys
import os

# Dodanie ścieżki do src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.db_connector import DatabaseConnector
from src.forecasting import Forecaster
from src.preprocessing import prepare_time_series, fill_missing_weeks

## 1. Pobranie Danych
Łączymy się z bazą i pobieramy historię zleceń.

In [ ]:
try:
    db = DatabaseConnector()
    print("Połączono z bazą!")
    
    df_hist = db.get_historical_data()
    print(f"Pobrano {len(df_hist)} wierszy historii.")
    
    # Preprocessing
    if not df_hist.empty:
        df_clean = prepare_time_series(df_hist)
        df_full = fill_missing_weeks(df_clean)
        print(f"Dane po przetworzeniu: {len(df_full)} wierszy (uzupełnione tygodnie).")
        display(df_full.head())
except Exception as e:
    print(f"Błąd: {e}")

## 2. Predykcja (Forecasting)
Użyjemy klasy `Forecaster` do wygenerowania prognozy różnymi metodami.

In [ ]:
forecaster = Forecaster()

if 'df_full' in locals() and not df_full.empty:
    # Wybierzmy jeden produkt do demo (jeśli istnieje)
    top_product = df_full['TowarId'].value_counts().index[0]
    print(f"Analiza dla produktu ID: {top_product}")
    
    df_prod = df_full[df_full['TowarId'] == top_product].copy()
    
    # 1. Random Forest
    preds_rf = forecaster.train_predict(df_prod, weeks_ahead=4, model_type='rf')
    print("\nRandom Forest Forecast:")
    if not preds_rf.empty: 
        display(preds_rf)

    # 2. Exponential Smoothing (Holt-Winters)
    preds_es = forecaster.train_predict(df_prod, weeks_ahead=4, model_type='es')
    print("\nExponential Smoothing Forecast:")
    if not preds_es.empty: 
        display(preds_es)
else:
    print("Brak danych do prognozy.")

## 3. Sprawdzenie BOM (Receptury)
Możemy też sprawdzić strukturę produktu (jeśli znamy ID wyrobu gotowego).

In [ ]:
try:
    # Przykładowe ID wyrobu (trzeba znać poprawne ID z bazy, np. z db.get_products_with_technology())
    # Można wylistować dostępne:
    # tech_prods = db.get_products_with_technology()
    # print(tech_prods.head())
    
    # example_id = 12345
    # bom = db.get_product_bom(example_id)
    # display(bom)
    pass
except Exceptionlist as e:
    print(e)